# Final Project, COMS 4995_005, Deep Learning

## Team Information

Team Member1 (Name,UNI): Richard Kennedy, rk2860

Python Version: 2
Utilizing Pytorch

In [2]:
import torch
import cPickle
import numpy as np
import matplotlib.pyplot as plt
from torch.autograd import Variable
import torch.nn as nn
import torch.nn.functional as F
import torchvision.models as models
import torchtext.vocab as Vocab

import torchvision.datasets as dset
import torchvision.transforms as transforms

import PIL
import random
import numpy


In [3]:
cap = dset.CocoCaptions(root = './train2014',
                        annFile = 'annotations/captions_train2014.json',
                        transform= transforms.Compose([transforms.Resize([224, 224]), transforms.ToTensor()]))

res = models.resnet50(pretrained=True)
vdim = 100
rSize = 2048
x = Vocab.GloVe(name='6B', dim= vdim)

paddSize =  rSize - vdim



loading annotations into memory...
Done (t=3.23s)
creating index...
index created!


In [4]:
a = x.vectors[x.stoi['man']]
print a.size()
b = torch.FloatTensor(paddSize).zero_()
paddedVec = torch.cat((a, b), 0)

torch.Size([100])


In [5]:
print "HELLO"
# img, target = cap[0]
# words = ['StopLSTM'] + target[0].split(' ') + ['EndLSTM']
# print words


# print wordVec(Vocab[words[1]])

HELLO


## Part 1

In [6]:
class LSTM(nn.Module):
    def __init__(self, embed_size, hidden_size, res50):
        super(LSTM, self).__init__()
        
        self.Esize = embed_size
        self.Hsize = hidden_size
        self.Wix = nn.Linear(embed_size, hidden_size)
        self.Wim = nn.Linear(hidden_size, hidden_size)
        self.Wfx = nn.Linear(embed_size, hidden_size)
        self.Wfm = nn.Linear(hidden_size, hidden_size)
        self.Wox = nn.Linear(embed_size, hidden_size)
        self.Wom = nn.Linear(hidden_size, hidden_size)
        self.Wcx = nn.Linear(embed_size, hidden_size)
        self.Wcm = nn.Linear(hidden_size, hidden_size)
        
        
        
        #res50 = torchvision.models.resnet50(pretrained=True)
        self.res50NoGrad = torch.nn.Sequential(*list(res50.children())[:-2])
        #Every layer besides the last layer does not need to be trained
        for param in self.res50NoGrad.parameters():
            param.requires_grad = False
        
        #set embed_size to dimensions of self.res50Last
        #output should be second to last layer
        self.res50Last = torch.nn.Sequential(list(res50.children())[-2])
        
        
        #self.glove = Vocab.GloVe(name='840B', dim= embed_size)
        
        

    def forward(self, x, m = None, c = None, CNN = False ):
        
        if not CNN:
            #glove requires x to be one hot word encoding?
            #x = self.glove(x)
            i = F.sigmoid(self.Wix(x)+self.Wim(m))
            f = F.sigmoid(self.Wfx(x)+self.Wfm(m))
            o = F.sigmoid(self.Wox(x)+self.Wom(m))
            h = F.tanh(self.Wcx(x) + self.Wcm(m))
            newC = f*c + i*h
            newM = o*newC
            #prediction dimension should line up with vocab_size
            #fine tune hidden dimension to make that work
            
            #log_softmax might not be necessary -> due to backprop
            
            #calculations: hidden dimensions = 50, embed_size = 2048, vocab_size not used
            #prediction = F.log_softmax(newM,)
            
            
            return newM, newC, newM
        
        else:
            x = self.res50NoGrad(x)
            x = self.res50Last(x)
            x = x.resize(1, self.Esize)
            i = F.sigmoid(self.Wix(x))
            f = F.sigmoid(self.Wfx(x))
            o = F.sigmoid(self.Wox(x))
            h = F.tanh(self.Wcx(x))
            c = i*h
            m = o*c
            
            return None, c, m

# net = LSTM(2048, 50, res)
# img, target = cap[0]
# img = img.unsqueeze(0)
# junk, c0, m0 =  net.forward(Variable(img), CNN = True)

# a = x.vectors[x.stoi['man']]
# b = torch.FloatTensor(1998).zero_()
# paddedVec = torch.cat((a, b), 0)

# pred, c0, m0 = net.forward(Variable(paddedVec), m = m0, c = c0)
# pred = pred.unsqueeze(0)

## Part 2

In [19]:
##Train


startW = 'start'
endW = 'stop'
## was lazy and used cat and dog as start and stop word. 
## Definetely going to have to change that
net = LSTM(rSize, vdim, res)
net = torch.load('0trainLSTM.pt')
import torch.optim as optim

criterion = nn.L1Loss()
optimizer = optim.SGD(filter(lambda p: p.requires_grad,net.parameters()), lr=0.001, momentum=.6)

counter = 0
reinitialize = True
for epoch in range(0, 1000):
    permute = np.random.permutation(len(cap))
    logLoss = 0.0
    numGoals = 0
    numGoalsT1 = 0
    numGoalsT2 = 0
    numGenerals = 0
    localMins = 0
    torch.save(net, str(epoch)+'LSTM.pt')
    
    for i in range(0, len(cap)):
#         img, target = cap[permute[i]]
        img, target = cap[i%20+10]
        words = [startW] + target[0].split('.')[0].split(' ') + [endW]
        
        length = len(words)
        
        img = img.unsqueeze(0)
        junk, c0, m0 =  net.forward(Variable(img), CNN = True)
        fullPred = None
        fullAnsw = None
        flip = True
        
        
        for j in range(0, length-1):
            lossPlus = 0
            #Going to need larger word corpus
            if words[j].lower() in x.stoi and words[j+1].lower() in x.stoi:
                
                a = x.vectors[x.stoi[words[j].lower()]]
                b = torch.FloatTensor(paddSize).zero_()
                paddedVec = torch.cat((a, b), 0)
                pred, c0, m0 = net.forward(Variable(paddedVec), m = m0, c = c0)
                
                
              

                answer = x.vectors[x.stoi[words[j+1].lower()]]
                answer = answer.unsqueeze(0)
                
               
                strKey = words[j+1].lower()
#                 if reinitialize:
#                     strKey = 'goals_none'
#                     answer = x.vectors[x.stoi['non-families'.lower()]]
#                     answer = answer.unsqueeze(0)
                
                
                #Trying to remove the Local Minimums
                dist1 = torch.dist(pred, Variable(answer)).data[0]
                dist2 = torch.dist(pred, Variable(x.vectors[x.stoi[random.choice(x.stoi.keys())]])).data[0]
                if abs(dist1 - dist2) <= .8 or dist1 >= dist2:
                    numGenerals+=1
                    
                if torch.dist(pred, Variable(x.vectors[x.stoi[strKey]])).data[0] <= 4:
                    
                    numGoals+=1
                    
                    if torch.dist(pred, Variable(x.vectors[x.stoi[strKey]])).data[0] <= 3:
                    
                        numGoalsT1+=1
                        
                        if torch.dist(pred, Variable(x.vectors[x.stoi[strKey]])).data[0] <= 2:
                    
                            numGoalsT2+=1
#                     answer = x.vectors[x.stoi['non-families'.lower()]]
#                     answer = answer.unsqueeze(0)
                    
                
               
                
#                 if dist1 < 3.5 and dist1 > 2.5:
#                     localMins+= 1
#                     lossPlus += .25
               
                
                
                loss = criterion(pred, Variable(answer))
                
                #loss.data[0]+=lossPlus
                loss.backward(retain_graph = True)
        
                optimizer.step()

                logLoss += loss.data[0]
                #logLoss += lossPlus
                if counter % 1000 == 999:    
                    print("EPOCH: "+str(epoch) + ", LOSS: " + str(logLoss/100))
                    print("Tight 4: "+ str(numGoals) + ", generals: "+str(numGenerals))
                    print ("Tight 3: " + str(numGoalsT1) +', Tight 4: '+str(numGoalsT2))
                    
                    numGoals = 0
                    numGenerals = 0
                    numGoalsT1 = 0
                    numGoalsT2 = 0
                    logLoss = 0.0
        
                if counter == 0:
                    print("EPOCH: "+str(epoch) + ", LOSS: " + str(logLoss))
                    print("goals_none: "+ str(numGoals) + ", generals: "+str(numGenerals))
                    
                counter+=1   

#                 if flip:
#                     fullPred = pred.unsqueeze(0)
#                     fullAnsw = answer.unsqueeze(0)
#                     flip = False
#                 else:
#                     answer = answer.unsqueeze(0)
#                     pred = pred.unsqueeze(0)
#                     fullPred = torch.cat((fullPred, pred), 0)
#                     fullAnsw = torch.cat((fullAnsw, answer), 0)
                    
        optimizer.zero_grad()         


        
#         loss = criterion(fullPred, Variable(fullAnsw))
#         loss.data[0]+=lossPlus
#         loss.backward()
        
#         optimizer.step()

#         logLoss += loss.data[0]
#         if counter % 100 == 99:    
#             print("EPOCH: "+str(epoch) + ", LOSS: " + str(logLoss/100))
#             logLoss = 0.0
        
#         if counter == 0:
#             print("EPOCH: "+str(epoch) + ", LOSS: " + str(logLoss))

#         counter+=1   
    
    

EPOCH: 0, LOSS: 0.0593814402819
goals_none: 1, generals: 0
EPOCH: 0, LOSS: 3.83510049302
Tight 4: 159, generals: 245
Tight 3: 66, Tight 4: 66
EPOCH: 0, LOSS: 3.62973484613
Tight 4: 186, generals: 221
Tight 3: 78, Tight 4: 67
EPOCH: 0, LOSS: 3.48856288608
Tight 4: 249, generals: 178
Tight 3: 92, Tight 4: 68
EPOCH: 0, LOSS: 3.39372594897
Tight 4: 303, generals: 159
Tight 3: 98, Tight 4: 67
EPOCH: 0, LOSS: 3.30913223397
Tight 4: 326, generals: 151
Tight 3: 118, Tight 4: 67
EPOCH: 0, LOSS: 3.22523202069
Tight 4: 358, generals: 140
Tight 3: 143, Tight 4: 67
EPOCH: 0, LOSS: 3.1833360743
Tight 4: 380, generals: 128
Tight 3: 160, Tight 4: 65
EPOCH: 0, LOSS: 3.110310464
Tight 4: 413, generals: 118
Tight 3: 171, Tight 4: 69


KeyboardInterrupt: 

## Extra Credit

In [17]:
torch.save(net, str(epoch)+'trainLSTM.pt')




#net = LSTM(rSize, vdim, res)

def createInverse(dic):
    new_dic ={}
    for key in dic:
        new_dic[dic[key]] = key
    
    return new_dic

invert = createInverse(x.stoi)

def closest(vec, n=1):
    """
    Find the closest words for a given vector
    """
    all_dists = [(w,torch.dist(vec, Variable(x.vectors[x.stoi[w]]))) for w in x.stoi]
    return sorted(all_dists, key=lambda t: t[1].data.numpy()[0])[:n]


img, target = cap[55]
print target
img = img.unsqueeze(0)
junk, c0, m0 =  net.forward(Variable(img), CNN = True)
a = x.vectors[x.stoi['start'.lower()]]
b = torch.FloatTensor(paddSize).zero_()
paddedVec = torch.cat((a, b), 0)
prediction, c0, m0 = net.forward(Variable(paddedVec), m = m0, c = c0)


strn = closest(prediction, 1)
print strn
#print whenNotSorted(prediction.data[0])
newWord = str(strn[0][0])
print newWord
#print strn
while True:
    a = x.vectors[x.stoi[newWord.lower()]]
    b = torch.FloatTensor(paddSize).zero_()
    paddedVec = torch.cat((a, b), 0)
    prediction, c0, m0 = net.forward(Variable(paddedVec), m = m0, c = c0)
    strn = closest(prediction, 1)
    newWord = str(strn[0][0])
    print newWord
    #print strn
    
    

[u'A tennis player is walking while holding his racket', u'Player and referee at tennis match on red court.', u'A man sitting on a high chair on a tennis court.', u"A tennis player walks in front of the moderator's stand.", u'A man on a court with a tennis racket.']
[(u'a', Variable containing:
 1.8085
[torch.FloatTensor of size 1]
)]
a
man
stands


KeyboardInterrupt: 

In [174]:

print closest(prediction, 100)

print strn[0]


import random

print torch.dist(Variable(x.vectors[x.stoi[random.choice(x.stoi.keys())]]), Variable(x.vectors[x.stoi['goals_none']]))

print torch.dist(Variable(x.vectors[x.stoi['snow']]), Variable(x.vectors[x.stoi['ice']]))


[(u'goals_none', Variable containing:
 2.6643
[torch.FloatTensor of size 1]
), (u'65stk', Variable containing:
 2.7356
[torch.FloatTensor of size 1]
), (u'__________________________________', Variable containing:
 2.7474
[torch.FloatTensor of size 1]
), (u'prohertrib', Variable containing:
 2.7507
[torch.FloatTensor of size 1]
), (u'http://www.oklahomacitynationalmemorial.org', Variable containing:
 2.7676
[torch.FloatTensor of size 1]
), (u'tissottiming.com', Variable containing:
 2.8126
[torch.FloatTensor of size 1]
), (u'____________________________________________', Variable containing:
 2.8153
[torch.FloatTensor of size 1]
), (u'timewrn', Variable containing:
 2.8185
[torch.FloatTensor of size 1]
), (u'_____________________________________________', Variable containing:
 2.8302
[torch.FloatTensor of size 1]
), (u'www.slarmy.org', Variable containing:
 2.8431
[torch.FloatTensor of size 1]
), (u'http://www.co.mo.md.us', Variable containing:
 2.8616
[torch.FloatTensor of size 1]
), (

In [131]:
print torch.dist(Variable(x.vectors[x.stoi['vegetables']]), Variable(x.vectors[x.stoi['well']]))

Variable containing:
 6.3827
[torch.FloatTensor of size 1]



In [42]:

print "HEY"
def farthest(vec, n=1):
    """
    Find the closest words for a given vector
    """
    all_dists = [(w,torch.dist(vec, Variable(x.vectors[x.stoi[w]]))) for w in x.stoi]
    return sorted(all_dists, key=lambda t: t[1].data.numpy()[0])[::-1][:n]

a = x.vectors[x.stoi['instance'.lower()]]

pred = farthest(Variable(a), 3)

print pred


HEY
[(u'non-families', Variable containing:
 13.4806
[torch.FloatTensor of size 1]
), (u'republish', Variable containing:
 12.1913
[torch.FloatTensor of size 1]
), (u'taiex', Variable containing:
 11.3278
[torch.FloatTensor of size 1]
)]


52
